codelab link: https://colab.research.google.com/github/abzzall/unsupervised_annotator1/blob/main/main.ipynb
github link: https://github.com/abzzall/unsupervised_annotator1.git



Install dependencies

In [1]:
!! pip install spacy 

['Collecting spacy',
 '  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/d6/9e/8afc618cfed4b5dc602b11754d4d9193a268439704defae301bffca7f04c/spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata',
 '  Downloading spacy-3.6.1-cp311-cp311-win_amd64.whl.metadata (26 kB)',
 'Collecting spacy-legacy<3.1.0,>=3.0.11 (from spacy)',
 '  Downloading spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)',
 'Collecting spacy-loggers<2.0.0,>=1.0.0 (from spacy)',
 '  Downloading spacy_loggers-1.0.4-py3-none-any.whl (11 kB)',
 'Collecting murmurhash<1.1.0,>=0.28.0 (from spacy)',
 '  Downloading murmurhash-1.0.9-cp311-cp311-win_amd64.whl (18 kB)',
 'Collecting cymem<2.1.0,>=2.0.2 (from spacy)',
 '  Downloading cymem-2.0.7-cp311-cp311-win_amd64.whl (28 kB)',
 'Collecting preshed<3.1.0,>=3.0.2 (from spacy)',
 '  Downloading preshed-3.0.8-cp311-cp311-win_amd64.whl (91 kB)',
 '     ---------------------------------------- 0.0/91.9 kB ? eta -:--:--',
 '     ---------------- -----

In [3]:
!! python -m spacy download en_core_web_sm

['Collecting en-core-web-sm==3.6.0',
 '  Downloading https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl (12.8 MB)',
 '     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--',
 '     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20',
 '     --------------------------------------- 0.0/12.8 MB 660.6 kB/s eta 0:00:20',
 '     --------------------------------------- 0.1/12.8 MB 409.6 kB/s eta 0:00:32',
 '     --------------------------------------- 0.1/12.8 MB 654.9 kB/s eta 0:00:20',
 '      -------------------------------------- 0.2/12.8 MB 841.6 kB/s eta 0:00:15',
 '      --------------------------------------- 0.3/12.8 MB 1.0 MB/s eta 0:00:13',
 '     - -------------------------------------- 0.4/12.8 MB 1.3 MB/s eta 0:00:10',
 '     - -------------------------------------- 0.6/12.8 MB 1.6 MB/s eta 0:00:08',
 '     -- ------------------------------------- 0.8/12.8 MB 2.1 MB/s et

In [1]:
import spacy 

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
text = "Using the part-of-speech tags we extract multiword expression candidates consisting of sequences of zero or more adjectives ADJ followed by nouns NOUN or proper nouns PROPN sequences."


In [6]:
doc=nlp(text)

In [10]:
# Initialize variables to store multiword expression candidates
candidates = []
current_candidate = []

# Define the allowed POS tags for adjectives and nouns
allowed_adjective_tags = {'ADJ'}
allowed_noun_tags = {'NOUN', 'PROPN'}

# Iterate through the tokens and their POS tags
for token in doc:
    if token.pos_ in allowed_adjective_tags or token.pos_ in allowed_noun_tags:
        # Append the token to the current candidate sequence
        current_candidate.append(token.text)
    else:
        # If the current token is not one of the desired POS tags, check if there's a valid candidate
        if current_candidate:
            candidates.append(" ".join(current_candidate))  # Join the tokens to form a candidate
            current_candidate = []  # Reset the current candidate

# Check if there's a valid candidate at the end of the text
if current_candidate:
    candidates.append(" ".join(current_candidate))

# Print the multiword expression candidates
for candidate in candidates:
    print(candidate)


part
speech tags
multiword expression candidates
sequences
more adjectives ADJ
nouns NOUN
proper nouns PROPN sequences


In [11]:
!!pip install sentence-transformers

['Collecting sentence-transformers',
 '  Downloading sentence-transformers-2.2.2.tar.gz (85 kB)',
 '     ---------------------------------------- 0.0/86.0 kB ? eta -:--:--',
 '     ------------- ------------------------ 30.7/86.0 kB 660.6 kB/s eta 0:00:01',
 '     ------------------ ------------------- 41.0/86.0 kB 393.8 kB/s eta 0:00:01',
 '     -------------------------------------- 86.0/86.0 kB 605.7 kB/s eta 0:00:00',
 '  Preparing metadata (setup.py): started',
 "  Preparing metadata (setup.py): finished with status 'done'",
 'Collecting transformers<5.0.0,>=4.6.0 (from sentence-transformers)',
 '  Obtaining dependency information for transformers<5.0.0,>=4.6.0 from https://files.pythonhosted.org/packages/e1/9d/4d9fe5c3b820db10773392ac5f4a0c8dab668f70b245ce2ce09785166128/transformers-4.33.0-py3-none-any.whl.metadata',
 '  Downloading transformers-4.33.0-py3-none-any.whl.metadata (119 kB)',
 '     ---------------------------------------- 0.0/119.9 kB ? eta -:--:--',
 '     --------